# **Salomon Uran Parra C.C. 1015068767**

## **Laboratorio 4 - Aprendizaje Estadistico**

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import sklearn as sk

### **1. Para simular un conjunto de características $x_1$ , $x_2$,..., $x_n$ trabajaremos en la primera parte con dos características de datos aleatorios que presentan un plano y mostraremos que los párametros optimizados se corresponden con el valor esperado.**

- Definir la ecuación  $y = 2.1*x_1 - 3.1*x_2$, y generar números aleatorios que pertenecen al plano.

- Realizar un diagrama 3D de los puntos generados aleatoriamente.


Nuestro objetivo será encontrar los valores $\theta_0 = 0, \theta_1=2.1, \theta_1=-3.1$ que mejor ajustar el plano, empleando cálculos vectorizados.

In [ ]:
#defino la ecuacion
y = lambda x1,x2: 2.1*x1 - 3.1*x2

#hago un sample aleatorio

np.random.seed(42)
#fijo la semilla para reproducibilidad de resultados

m = 1000
#numero de datos

x1 = (np.random.random(m) - 0.5)*2
x2 = (np.random.random(m) - 0.5)*2
#numero aleatorio de m puntos en R2

Y = y(x1,x2)
#funcion evaluada en los m datos


fig = go.Figure(data=[go.Scatter3d(x=x1, y=x2, z=Y, mode='markers',
                                   marker=dict(
                                       size=5,
                                       color=Y,
                                       colorscale='viridis',
                                       opacity=0.8
                                   ))])
fig.update_layout(title='Grafico 3D de la funcion',
                     scene = dict(
                         xaxis_title='x1',
                         yaxis_title='x2',
                         zaxis_title='y'),
                     margin=dict(l=0, r=0, b=0, t=40))
fig.show()

### **2. Inicializar conjunto de parámetros $\Theta$ de manera aleatoria.**

In [ ]:
#inicializacion de los parametros aleatoriamente

n = 3 #numero de features mas 1

theta = (np.random.random(n)-0.5)*10
#theta aleatorio entre -5 y 5

theta = np.matrix(theta)
#se convierte en matriz

print(theta.shape)
print(theta)

### **3. Construir la matrix X con dimensiones $(n+1, m)$, m es el numero de datos de entrenamiento y (n) el número de caracteristicas.**

In [ ]:
#construccion de la matriz X (n,m) (aqui n es n+1)

X = np.matrix([np.ones(len(x1)),x1,x2])

X.shape

### **4. Calcular la función de coste(revise cuidosamente las dimensiones de cada matriz):**

  - $h = \Theta^{T} X $
  - $\Lambda= (h -Y) $
  - $\Lambda*= (h -Y)^2 $
  - $\Lambda= [\Lambda_1,\Lambda_2, ...,\Lambda_m]$
  - $J = \frac{1}{2m} \sum_{i}^m \Lambda_i $

In [ ]:
h = theta @ X
#primer calculamos h que es la prediccion inicial, recordar que theta es (1,3) y X es (3,1000), por lo que h es (1,1000)

Y = np.matrix(Y)
Y.shape
#Y tambien es (1,1000)

lda = h - Y
#ahora calculamos la diferencia entre la prediccion y los valores esperados, (1,1000)

ldaa = np.square(lda)
#ahora calculamos el cuadrado de las diferencias, esto es (1,1000)

J = (1/(2*m))*np.sum(ldaa)
#ahora calculamos la funcion de coste sumando sobre ldaa

print(J)

### **5. Aplicar el gradiente descendente:**
  - Encontrar el gradiente.
  $$\nabla J = \frac{1}{m}\Lambda X.T$$
  
  - Actualizar los nuevos parametros:
  $$\Theta_{n+1}=\Theta_{n}-\alpha\nabla J$$

In [ ]:
DJ = lda @ X.T *1/m
#calculamos el gradiente como el vector de las diferencias entre h y Y, multiplicado por la matriz transpuesta de los features
#aqui recordar que lda es (1,1000) y X.T es (1000,3), luego DJ es (1,3)

print('Gradiente de J actual: ',DJ)

#se actualizan los nuevos parametros dado un alpha = 0.1
alpha = 0.1

theta = theta - alpha*DJ
#algoritmo de gradiente descendente

print('Siguiente theta: ',theta)

### **6. Iterar para encontrar los valores $\Theta$ que se ajustan el plano.**

In [ ]:
#para poder iterar vamos a definir funciones utiles primero

ndata = 1000
nfeat = 2
k1 = 10
#numero de datos m, numero de features n, amplitud del intervalo de inicializacion de los features X, k1

y = lambda x1,x2: 2.1*x1 - 3.1*x2
#funcion a fittear, es decir los labels o targets

theta = np.matrix(np.random.random(nfeat+1))
#inicializo los parametros


#la funcion que arroja la prediccion
def modelo_lineal(nfeat, ndata, thetax, k = 2, seed = 42):
  np.random.seed(seed)
  #reproducibilidad

  arr = [np.ones(ndata)]
  #se inicializa la matriz de datos con la fila de 1's

  for i in range(nfeat):
    arr.append((np.random.random(ndata)-0.5)*k)
    #se toman m datos aleatorios entre -k/2 y k/2 para el i-esimo feature

  X = np.matrix(arr)
  #se vuelve una matriz que debe ser (n+1,m)

  return X

X = modelo_lineal(nfeat,ndata,theta,k = k1)
#primera prediccion

Y = y(X[1],X[2])
#se evaluan los datos

Y = np.matrix(Y)
#matriz (1,m)


#se define la funcion coste como antes
def coste(theta, X, Y):
  h = theta @ X
  #primera prediccion entre theta y X, donde h es (1,1000)

  m = X.shape[1]
  #numero de datos, osea dimension de columnas de X

  lda = h - Y
  #diferencia entre prediccion y target

  ldaa = np.square(lda)
  #matriz (1,1000) que son los cuadrados de las componentes de la diferencia

  J = (1/(2*m))*np.sum(ldaa)
  #se promedia el vector de los cuadrados

  DJ = lda @ X.T *1/m
  #se obtiene el gradiente de J en la prediccion, que es (1,1000)x(1000,3), osea (1,3)

  return J, DJ

alpha = 0.1
#se define un alfa

eps = 1e-7
#una tolerancia

#como la fucion de coste para el modelo multilineal es un paraboloide en 3 dimensiones de theta, se puede esperar
#de manera analoga a la funcion en 2 dimensiones, que su minimo sea cuando se anule la funcion de coste

for i in range(300):
  #se define un numero de epochs
  J, DJ = coste(theta,X,Y)

  if J<eps:
    break
    #condicion de tolerancia

  theta = theta - alpha*DJ
  #gradiente descendente

print(theta)


### **7. Reescribir su código como una clase.**

In [ ]:
#se define la clase multilinearregresion
class MultilinearRegresion():
  def __init__(self, X, Y,n):
    self.X = X
    self.Y = Y
    Nfeatures = np.shape(X)[0]
    m = np.shape(X)[1]
    self.theta = np.matrix(np.random.random(Nfeatures).reshape(Nfeatures, 1))
  #se definen en la clase los elementos X, Y, Nfeatures, m, y se inicializa theta

  def model(self):
    self.h = self.theta.T@self.X
  #se define la funcion model() que crea la prediccion h que es una matriz (1,m)


  def costo(self):
    self.J =  np.mean(np.square(self.h - self.Y))
  #se define la funcion costo() que crea el valor de J evaluado en la prediccion

  def update_params(self, learning_rate):
    """
    Gradiente descendente
    """
    self.grad = (self.h - self.Y)@self.X.T/m#
    self.theta = self.theta - learning_rate*self.grad.T#...
  #se define la funcion update_params() que calcula el gradiente de J respecto los parametros, que es (1,1000)x(1000,3) = (1,3)
  #y que actualiza el valor de theta por medio del gradiente descendente

  def fit(self, learning_rate):
    for i in range(0, n):
      self.model()
      self.costo()
      self.update_params(learning_rate)
    return self.theta
  #se define la funcion fit() que itera sobre una cantidad de epochs, n, la prediccion, la funcion de costo, y la actualizacion de parametros, y retorna al final los parametros theta

In [ ]:
n = 500
#se define un numero de epochs

regresion = MultilinearRegresion(X,Y,n)
#se define el modelo de regresion

theta = regresion.fit(0.1)
#se ejecuta la funcion fit() conn learning_rate = 0.1

print('El conjunto de parametros theta0, theta1 y theta2 obtenidos es: ',theta)

Asi, vemos que tanto la clase como la iteracion del punto 6 recuperan aproximadamente (dependiendo del numero de epochs n), el conjunto de parametros que dan el target $y$, los cuales son $\theta_0 = 0, \theta_1 = 2.1, \theta_2 = -3.1$